<a href="https://colab.research.google.com/github/PurnaPritam/Role-Identifier-AI/blob/main/Role-Identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
import random
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
import torch.nn as nn


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

csv_file_location = '/content/train (2).csv'
data = pd.read_csv(csv_file_location)
category = data['Category']
text = data['Text']
category_arr = np.array(category)

category_names = list(set(category_arr))
category_names.sort()
category_names = np.array(category_names)
print("Categories:", category_names)


In [ ]:

stop_words = set(stopwords.words('english'))
stop_words_upper = {w.upper() for w in stop_words}
all_stop_words = stop_words.union(stop_words_upper)

tokenized = []
for t in text:
    tokens = word_tokenize(t)
    tokens = [word for word in tokens if word not in all_stop_words]
    tokenized.append(tokens)

lemmatizer = WordNetLemmatizer()

def tagger(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

lemmatized_text = []
for tokens in tokenized:
    tags = nltk.pos_tag(tokens)
    lemmatized_tokens = []
    for word, pos in tags:
        wn_tag = tagger(pos)
        if wn_tag is None:
            lemmatized_tokens.append(word)
        else:
            lemmatized_tokens.append(lemmatizer.lemmatize(word, wn_tag))
    lemmatized_text.append(" ".join(lemmatized_tokens))

In [ ]:


random.seed(10)
random.shuffle(lemmatized_text)
random.seed(10)
random.shuffle(category_arr)

n = int(len(lemmatized_text) * 0.8)
train_texts, val_texts = lemmatized_text[:n], lemmatized_text[n:]
train_labels_raw, val_labels_raw = category_arr[:n], category_arr[n:]

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_enc = tokenizer(train_texts, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
val_enc = tokenizer(val_texts, padding='max_length', max_length=512, truncation=True, return_tensors="pt")

encoder = LabelEncoder()
train_labels = torch.tensor(encoder.fit_transform(train_labels_raw))
val_labels = torch.tensor(encoder.transform(val_labels_raw))

batch_size = 32

train_dataset = TensorDataset(train_enc['input_ids'], train_enc['attention_mask'], train_labels)
val_dataset = TensorDataset(val_enc['input_ids'], val_enc['attention_mask'], val_labels)

train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size)

model_0 = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(category_names)
)

In [ ]:


for param in model_0.distilbert.parameters():
    param.requires_grad = False

for param in model_0.classifier.parameters():
    param.requires_grad = True

optimizer = AdamW(model_0.parameters(), lr=1e-4)

def evaluate(model, loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
        for batch in loader:
            input_ids, attention_mask, labels = batch
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    return correct / total

In [ ]:

epochs = 3
for epoch in range(epochs):
    model_0.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        outputs = model_0(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} | Train Acc: {evaluate(model_0, train_dataloader):.4f} | Val Acc: {evaluate(model_0, val_dataloader):.4f}")

for param in model_0.parameters():
    param.requires_grad = True

optimizer = AdamW(model_0.parameters(), lr=5e-5)

epochs = 3
for epoch in range(epochs):
    model_0.train()
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        outputs = model_0(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"[Fine-tune] Epoch {epoch+1} | Train Acc: {evaluate(model_0, train_dataloader):.4f} | Val Acc: {evaluate(model_0, val_dataloader):.4f}")